In [112]:
from pyspark.sql import *
from pyspark.sql.functions import *


In [113]:
spark = SparkSession.builder \
        .appName("05-working_with_df") \
        .master("local[*]") \
        .getOrCreate()

In [114]:
raw_fire_df = spark.read \
            .format("csv") \
            .option("header", True) \
            .option("inferSchema", True) \
            .load(r"C:\Users\chasurag\Documents\my repos\Fire_Department_and_Emergency_Medical_Services_Dispatched_Calls_for_Service_20250806.csv")

In [116]:
raw_fire_df.show(2)

+-----------+-------+---------------+----------------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+-------------+-------------------+---------+------------+----+-----------------+--------+--------------+--------+--------------------+----------------+---------+------------------------------+------------------------+-------------------+------------------------------------+--------------+--------------------+--------------------+--------------------+
|Call Number|Unit ID|Incident Number|       Call Type| Call Date|Watch Date|       Received DtTm|          Entry DtTm|       Dispatch DtTm|       Response DtTm|       On Scene DtTm|      Transport DtTm|       Hospital DtTm|Call Final Disposition|      Available DtTm|             Address|         City|Zipcode of Incident|Battalion|Station Area| Box|Original 

In [117]:
# renamed_fire_df = raw_fire_df \
#     .withColumnRenamed("Call Number", 'callnumber') \
#     .withColumnRenamed('Unit Id', 'unitid')

renamed_fire_df = raw_fire_df \
    .withColumnsRenamed({"Call Number":"CallNumber", "Unit ID":"UnitID", "Incident Number":"IncidentNumber", "Call Type":"CallType", "Call Date":"CallDate", "Watch Date":"WatchDate", "Received DtTm":"ReceivedDtTm", "Entry DtTm":"EntryDtTm", "Dispatch DtTm":"DispatchDtTm", "Response DtTm":"ResponseDtTm", "On Scene DtTm":"OnSceneDtTm", "Transport DtTm":"TransportDtTm", "Hospital DtTm":"HospitalDtTm", "Call Final Disposition":"CallFinalDisposition", "Available DtTm":"AvailableDtTm", "Address":"Address", "City":"City", "Zipcode of Incident":"ZipcodeofIncident", "Battalion":"Battalion", "Station Area":"StationArea", "Box":"Box", "Original Priority":"OriginalPriority", "Priority":"Priority", "Final Priority":"FinalPriority", "ALS Unit":"ALSUnit", "Call Type Group":"CallTypeGroup", "Number of Alarms":"NumberofAlarms", "Unit Type":"UnitType", "Unit sequence in call dispatch":"Unitsequenceincalldispatch", "Fire Prevention District":"FirePreventionDistrict", "Supervisor District":"SupervisorDistrict", "Neighborhooods - Analysis Boundaries":"NeighborhooodsAnalysisBoundaries", "RowID":"RowID", "case_location":"case_location", "data_as_of":"data_as_of", "data_loaded_at":"data_loaded_at"})

renamed_fire_df.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----------------+---------+-----------+----+----------------+--------+-------------+-------+--------------------+--------------+--------+--------------------------+----------------------+------------------+--------------------------------+---------------+--------------------+--------------------+--------------------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|        ReceivedDtTm|           EntryDtTm|        DispatchDtTm|        ResponseDtTm|         OnSceneDtTm|       TransportDtTm|        HospitalDtTm|CallFinalDisposition|       AvailableDtTm|             Address|         City|ZipcodeofIncident|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|A

In [118]:
temp_fire_df = renamed_fire_df \
            .withColumn('CallDate', to_date('CallDate', 'MM/dd/yyyy')) \
            .withColumn('WatchDate', to_date('WatchDate', 'MM/dd/yyyy')) \
            .withColumn('ReceivedDtTm', to_timestamp('ReceivedDtTm', 'MM/dd/yyyy hh:mm:ss a')) \
            .withColumn('EntryDtTm', to_timestamp('EntryDtTm', 'MM/dd/yyyy hh:mm:ss a')) \
            .withColumn('DispatchDtTm', to_timestamp('DispatchDtTm', 'MM/dd/yyyy hh:mm:ss a')) \
            .withColumn('ResponseDtTm', to_timestamp('ResponseDtTm', 'MM/dd/yyyy hh:mm:ss a')) \
            .withColumn('OnSceneDtTm', to_timestamp('OnSceneDtTm', 'MM/dd/yyyy hh:mm:ss a')) \
            .withColumn('TransportDtTm', to_timestamp('TransportDtTm', 'MM/dd/yyyy hh:mm:ss a')) \
            .withColumn('HospitalDtTm', to_timestamp('HospitalDtTm', 'MM/dd/yyyy hh:mm:ss a')) \
            .withColumn('AvailableDtTm', to_timestamp('AvailableDtTm', 'MM/dd/yyyy hh:mm:ss a'))

temp_fire_df.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: date (nullable = true)
 |-- WatchDate: date (nullable = true)
 |-- ReceivedDtTm: timestamp (nullable = true)
 |-- EntryDtTm: timestamp (nullable = true)
 |-- DispatchDtTm: timestamp (nullable = true)
 |-- ResponseDtTm: timestamp (nullable = true)
 |-- OnSceneDtTm: timestamp (nullable = true)
 |-- TransportDtTm: timestamp (nullable = true)
 |-- HospitalDtTm: timestamp (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: timestamp (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ZipcodeofIncident: double (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: integer (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable 

In [119]:
fire_df = temp_fire_df.withColumn("delay", 
                                     (unix_timestamp(col('responsedttm')) - unix_timestamp(col('ReceivedDtTm'))) / 60 ) \
                                     .withColumn('delay', round('delay', 1))


fire_df.select('delay').show(5)

+-----+
|delay|
+-----+
|  4.9|
|  2.5|
|  2.6|
|  4.2|
| 11.3|
+-----+
only showing top 5 rows



In [53]:
fire_df.cache()

DataFrame[CallNumber: int, UnitID: string, IncidentNumber: int, CallType: string, CallDate: date, WatchDate: date, ReceivedDtTm: timestamp, EntryDtTm: timestamp, DispatchDtTm: timestamp, ResponseDtTm: timestamp, OnSceneDtTm: timestamp, TransportDtTm: timestamp, HospitalDtTm: timestamp, CallFinalDisposition: string, AvailableDtTm: timestamp, Address: string, City: string, ZipcodeofIncident: double, Battalion: string, StationArea: int, Box: string, OriginalPriority: string, Priority: string, FinalPriority: int, ALSUnit: boolean, CallTypeGroup: string, NumberofAlarms: int, UnitType: string, Unitsequenceincalldispatch: int, FirePreventionDistrict: string, SupervisorDistrict: string, NeighborhooodsAnalysisBoundaries: string, RowID: string, case_location: string, data_as_of: string, data_loaded_at: string, delay: double]

In [ ]:
## Q1. How many distinct types of calls were made to the Fire Department?

# select count(distinct CallType) as distinct_call_type_count
# from fire_service_calls_tbl
# where CallType is not null


In [57]:
q1_df = fire_df.where('calltype is not null') \
                .select('calltype') \
                .distinct()

q1_df.count()

30

In [58]:
## Q2. What were distinct types of calls made to the Fire Department?

# select distinct CallType as distinct_call_types
# from fire_service_calls_tbl
# where CallType is not null


In [64]:
q2_df = fire_df.where('calltype is not null') \
                .select(expr('calltype as distinct_call_type')) \
                .distinct()

q2_df.show(truncate=False)

+--------------------------------------------+
|distinct_call_type                          |
+--------------------------------------------+
|Elevator / Escalator Rescue                 |
|Structure Fire / Smoke in Building          |
|Administrative                              |
|Alarms                                      |
|Odor (Strange / Unknown)                    |
|Citizen Assist / Service Call               |
|HazMat                                      |
|Watercraft in Distress                      |
|Explosion                                   |
|Vehicle Fire                                |
|Extrication / Entrapped (Machinery, Vehicle)|
|Other                                       |
|Outside Fire                                |
|Traffic Collision                           |
|Assist Police                               |
|Gas Leak (Natural and LP Gases)             |
|Water Rescue                                |
|Electrical Hazard                           |
|Industrial A

In [61]:
## Q3. Find out all response for delayed times greater than 5 mins?

# select CallNumber, Delay
# from fire_service_calls_tbl
# where Delay > 5

In [65]:
q3_df = fire_df.where('delay > 5') \
                .select('callnumber', expr('delay as delay_min'))

q3_df.show()

+----------+---------+
|callnumber|delay_min|
+----------+---------+
| 240022253|     11.3|
| 240020144|      5.1|
| 240021192|     10.0|
| 240021851|      6.5|
| 240020098|      5.5|
| 240022004|      5.4|
| 240020492|     12.6|
| 240021429|     10.8|
| 240020464|      6.1|
| 240021366|      6.7|
| 240020201|      5.2|
| 240022622|      7.0|
| 240020889|     32.8|
| 240022175|     12.5|
| 240022317|      6.5|
| 240022409|      5.7|
| 240022312|      5.2|
| 240021201|      5.8|
| 240021956|     20.3|
| 240021868|      5.4|
+----------+---------+
only showing top 20 rows



In [66]:
## Q4. What were the most common call types?

# select CallType, count(*) as count
# from fire_service_calls_tbl
# where CallType is not null
# group by CallType
# order by count desc


In [ ]:
q4_df = fire_df.where('calltype is not null') \
                .select('calltype') \
                .groupBy('calltype') \
                .count() \
                .orderBy('count', ascending=False)

q4_df.show()

+--------------------+------+
|            calltype| count|
+--------------------+------+
|    Medical Incident|390175|
|              Alarms| 82230|
|Structure Fire / ...| 29152|
|   Traffic Collision| 18883|
|               Other| 18365|
|Citizen Assist / ...| 12426|
|        Outside Fire| 11371|
|Gas Leak (Natural...|  4460|
|        Water Rescue|  3343|
|   Electrical Hazard|  3077|
|Elevator / Escala...|  1954|
|        Vehicle Fire|  1602|
|Smoke Investigati...|  1206|
|Odor (Strange / U...|   430|
|          Fuel Spill|   387|
|      Administrative|   184|
|Extrication / Ent...|   174|
|Train / Rail Inci...|   152|
|           Explosion|   139|
|              HazMat|   130|
+--------------------+------+
only showing top 20 rows



In [78]:
## Q5. What zip codes accounted for most common calls?

# select CallType, ZipCode, count(*) as count
# from fire_service_calls_tbl
# where CallType is not null
# group by CallType, Zipcode
# order by count desc


In [104]:
fire_df.select('calltype', expr('cast(ZipcodeofIncident as int) as zipcode')) \
        .where('calltype is not null') \
        .groupBy('calltype', 'zipcode') \
        .count() \
        .orderBy('count', ascending = False) \
        .show()

+----------------+-------+-----+
|        calltype|zipcode|count|
+----------------+-------+-----+
|Medical Incident|  94103|65937|
|Medical Incident|  94102|54461|
|Medical Incident|  94109|41521|
|Medical Incident|  94110|29729|
|Medical Incident|  94124|19160|
|Medical Incident|  94112|17418|
|Medical Incident|  94115|14116|
|Medical Incident|  94107|12661|
|Medical Incident|  94122|12412|
|          Alarms|  94102|10765|
|Medical Incident|  94133|10722|
|Medical Incident|  94105|10700|
|Medical Incident|  94114|10096|
|Medical Incident|  94117| 9853|
|          Alarms|  94103| 9816|
|Medical Incident|  94134| 9734|
|Medical Incident|  94118| 9219|
|          Alarms|  94109| 9120|
|Medical Incident|  94132| 8488|
|Medical Incident|  94116| 8377|
+----------------+-------+-----+
only showing top 20 rows



In [86]:
## Q6. What San Francisco neighborhoods are in the zip codes 94102 and 94103

# select distinct Neighborhood, Zipcode
# from fire_service_calls_tbl
# where Zipcode== 94102 or Zipcode == 94103


In [93]:
# fire_df.where('zipcodeofincident in (94102, 94103)') \
#         .select('address', col('zipcodeofincident').cast('int').alias('zipcode')) \
#         .distinct() \
#         .show(truncate = False)

# or

fire_df.where('zipcodeofincident == 94102 or zipcodeofincident == 94103') \
        .select('address', col('zipcodeofincident').cast('int').alias('zipcode')) \
        .distinct() \
        .show(truncate = False)

+--------------------------+-------+
|address                   |zipcode|
+--------------------------+-------+
|JESSIE ST/MINT PLZ/MINT ST|94103  |
|OCTAVIA ST/HAYES ST       |94102  |
|VERMONT ST/DIVISION ST    |94103  |
|TAYLOR ST/OPAL PL         |94102  |
|TEHAMA ST/08TH ST         |94103  |
|ROSE ST/GOUGH ST          |94102  |
|08TH ST/MISSION ST        |94103  |
|HAYES ST/GOUGH ST         |94102  |
|FRANKLIN ST/FULTON ST     |94102  |
|VALENCIA ST/16TH ST       |94103  |
|VALENCIA ST/CLINTON PARK  |94103  |
|LAFAYETTE ST/MINNA ST     |94103  |
|OAK ST/BUCHANAN ST        |94102  |
|11TH ST/KISSLING ST       |94103  |
|PAGE ST/LAGUNA ST         |94102  |
|NATOMA ST/08TH ST         |94103  |
|FULTON ST/LAGUNA ST       |94102  |
|TRAINOR ST/14TH ST        |94103  |
|BRYANT ST/LANGTON ST      |94103  |
|CLINTON PARK/DOLORES ST   |94103  |
+--------------------------+-------+
only showing top 20 rows



In [94]:
## Q7. What was the sum of all calls, average, min and max of the response times for calls?

# select sum(NumAlarms), avg(Delay), min(Delay), max(Delay)
# from fire_service_calls_tbl


In [100]:
fire_df.select(
        sum('NumberofAlarms').alias('Total Alarms'), 
        avg('delay').alias('Average Delay'), 
        min('delay').alias('Minimum Delay'), 
        max('delay').alias("Maximum Delay")
        ).show()

+------------+----------------+-------------+-------------+
|Total Alarms|   Average Delay|Minimum Delay|Maximum Delay|
+------------+----------------+-------------+-------------+
|      581330|5.15148044915186|        -90.1|      10385.6|
+------------+----------------+-------------+-------------+



In [102]:
## Q8. How many distinct years of data is in the CSV file?

# select distinct year(to_timestamp(CallDate, "MM/dd/yyyy")) as year_num
# from fire_service_calls_tbl
# order by year_num

In [103]:
fire_df.select(year('calldate').alias('year_of_call_date')) \
        .distinct() \
        .orderBy('year_of_call_date') \
        .show()

+-----------------+
|year_of_call_date|
+-----------------+
|             2024|
|             2025|
+-----------------+



In [105]:
## Q9. What week of the year in 2018 had the most fire calls?

# select weekofyear(to_timestamp(CallDate, "MM/dd/yyyy")) week_year, count(*) as count
# from fire_service_calls_tbl 
# where year(to_timestamp(CallDate, "MM/dd/yyyy")) == 2018
# group by week_year
# order by count desc

In [107]:
fire_df.where(year('calldate') == 2025) \
        .select(weekofyear('calldate').alias('week_year')) \
        .groupBy('week_year') \
        .count() \
        .orderBy('count', ascending = False) \
        .show()

+---------+-----+
|week_year|count|
+---------+-----+
|        6| 8278|
|        7| 7786|
|        3| 7630|
|        4| 7495|
|        2| 7376|
|        5| 7299|
|        9| 7157|
|       11| 7149|
|       13| 7133|
|       14| 7051|
|       26| 7032|
|        8| 7028|
|       18| 6998|
|       19| 6987|
|       21| 6982|
|       16| 6961|
|       27| 6944|
|       23| 6929|
|       22| 6879|
|       12| 6834|
+---------+-----+
only showing top 20 rows



In [108]:
## Q10. What neighborhoods in San Francisco had the worst response time in 2018?

# select Neighborhood, Delay
# from fire_service_calls_tbl 
# where year(to_timestamp(CallDate, "MM/dd/yyyy")) == 2018


In [111]:
fire_df.where(year(col('calldate')) == 2025) \
        .select('address', 'delay') \
        .orderBy('delay', ascending = False) \
        .show(truncate = False)

+-----------------------------+-------+
|address                      |delay  |
+-----------------------------+-------+
|CALL BOX: 2310 FOLSOM ST,SF  |10385.6|
|CALL BOX: 2310 FOLSOM ST,SF  |10385.6|
|CALL BOX: BUCHANAN ST/TURK ST|10099.8|
|CALL BOX: BUCHANAN ST/TURK ST|10097.9|
|CALL BOX: 2310 FOLSOM ST,SF  |5836.2 |
|CALL BOX: 6TH ST/MISSION ST  |2592.4 |
|CALL BOX: 6TH ST/MISSION ST  |2340.8 |
|CALL BOX: 6TH ST/MISSION ST  |2340.5 |
|CALL BOX: 6TH ST/MISSION ST  |2120.0 |
|CALL BOX: 6TH ST/MISSION ST  |1911.4 |
|CALL BOX: 6TH ST/MISSION ST  |1444.3 |
|CALL BOX: 6TH ST/MISSION ST  |1420.3 |
|CALL BOX: 6TH ST/MISSION ST  |1397.3 |
|CALL BOX: 6TH ST/MISSION ST  |1396.8 |
|CALL BOX: 6TH ST/MARKET ST   |1391.0 |
|CALL BOX: 6TH ST/MARKET ST   |1385.9 |
|CALL BOX: 6TH ST/MISSION ST  |1347.3 |
|CALL BOX: 6TH ST/MISSION ST  |1314.6 |
|CALL BOX: 6TH ST/MISSION ST  |1311.1 |
|CALL BOX: 6TH ST/MISSION ST  |1300.7 |
+-----------------------------+-------+
only showing top 20 rows

